In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re
from sklearn.model_selection import train_test_split

In [13]:
df = pd.read_csv(r'C:\Users\ALI\itmo-cpp\input_data\cpps_for_regression.csv')

In [14]:
sequences_df = df[['sequence']].copy() # Используем .copy() для избежания SettingWithCopyWarning


In [15]:
sequences_df['sequence'] = sequences_df['sequence'].astype(str).str.upper()


In [16]:
allowed_chars = set("ACDEFGHIKLMNPQRSTVWYXZBJ")

def is_valid_sequence(seq, allowed_set):
    """Проверяет, состоит ли последовательность только из разрешенных символов."""
    if not isinstance(seq, str) or not seq: # Проверка на пустую строку или не строку
        return False
    return all(char in allowed_set for char in seq)

# Применяем фильтрацию
sequences_df['is_valid'] = sequences_df['sequence'].apply(lambda x: is_valid_sequence(x, allowed_chars))
filtered_sequences_df = sequences_df[sequences_df['is_valid']].copy()
filtered_sequences_df = filtered_sequences_df.drop(columns=['is_valid']) # Удаляем вспомогательный столбец

In [17]:
filtered_sequences_df

,sequence
22,AAAWFW
23,AAKFIKKRRTKSS
24,AAKLLPDLLAAP
25,AAVACRICMRNFSTRQARRNHRRRHRR
26,AAVALLPAVLLALLAK
...,...
1593,VCVR
1594,VLCLR
1595,VPTLK
1596,VRLPPPVRLPPPVRLPPP


In [ ]:
intermediate_csv_path = 'input_data/cpps_for_generator.csv'
filtered_sequences_df.to_csv(intermediate_csv_path, index=False)

In [20]:
train_seqs, val_seqs = train_test_split(filtered_sequences_df['sequence'].tolist(), test_size=0.1, random_state=42)

In [26]:
print(len(train_seqs))
print(len(val_seqs))

1191
133


In [21]:
def save_protgpt2_format(seqs, filename):
    with open(filename, 'w') as f:
        for seq in seqs:
            f.write("<|endoftext|>\n")
            # Разбиваем длинные строки на 80 символов, как в FASTA
            for i in range(0, len(seq), 80):
                f.write(seq[i:i+80] + '\n')

In [22]:
# === 7. Сохраняем train и validation ===
save_protgpt2_format(train_seqs, "train.txt")
save_protgpt2_format(val_seqs, "valid.txt")